- 사용자가 특정 영화를 감상하고 좋아했다면 그 영화와 비슷한 특성/속성, 구성 요소 등을 가진 다른 영화 추천
- 데이터 : https://www.kaggle.com/tmdb/tmdb-movie-metadata

In [2]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')
import os
os.listdir('./data/tmdb')

['tmdb_5000_credits.csv', 'tmdb_5000_movies.csv']

In [4]:
movies = pd.read_csv('./data/tmdb/tmdb_5000_movies.csv')
movies.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


# 데이터 전처리

In [5]:
movies_df = movies[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]

In [6]:
pd.set_option('max_colwidth',100)

In [7]:
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


In [8]:
#string을  list[dict1, dict2] 객체로
from ast import literal_eval

In [9]:
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [10]:
movies_df['genres'][0]

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [11]:
movies_df['keywords'][0]

[{'id': 1463, 'name': 'culture clash'},
 {'id': 2964, 'name': 'future'},
 {'id': 3386, 'name': 'space war'},
 {'id': 3388, 'name': 'space colony'},
 {'id': 3679, 'name': 'society'},
 {'id': 3801, 'name': 'space travel'},
 {'id': 9685, 'name': 'futuristic'},
 {'id': 9840, 'name': 'romance'},
 {'id': 9882, 'name': 'space'},
 {'id': 9951, 'name': 'alien'},
 {'id': 10148, 'name': 'tribe'},
 {'id': 10158, 'name': 'alien planet'},
 {'id': 10987, 'name': 'cgi'},
 {'id': 11399, 'name': 'marine'},
 {'id': 13065, 'name': 'soldier'},
 {'id': 14643, 'name': 'battle'},
 {'id': 14720, 'name': 'love affair'},
 {'id': 165431, 'name': 'anti war'},
 {'id': 193554, 'name': 'power relations'},
 {'id': 206690, 'name': 'mind and soul'},
 {'id': 209714, 'name': '3d'}]

In [12]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [y['name'] for y in x])
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


In [15]:
#list
movies_df['genres'][0:5]

0    [Action, Adventure, Fantasy, Science Fiction]
1                     [Adventure, Fantasy, Action]
2                       [Action, Adventure, Crime]
3                 [Action, Crime, Drama, Thriller]
4             [Action, Adventure, Science Fiction]
Name: genres, dtype: object

# genre contents similarity

## genre 칼럼을 문자열로 변환

In [16]:
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
movies_df['genres_literal'][:5]

0    Action Adventure Fantasy Science Fiction
1                    Adventure Fantasy Action
2                      Action Adventure Crime
3                 Action Crime Drama Thriller
4            Action Adventure Science Fiction
Name: genres_literal, dtype: object

In [19]:
movies_df.shape

(4803, 9)

## 피처 벡터화 행렬로 변환

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(min_df = 0, ngram_range=(1,2)) #count기반
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
genre_mat.shape

(4803, 276)

In [29]:
count_vect.vocabulary_

{'action': 0,
 'adventure': 16,
 'fantasy': 124,
 'science': 232,
 'fiction': 138,
 'action adventure': 1,
 'adventure fantasy': 24,
 'fantasy science': 135,
 'science fiction': 233,
 'fantasy action': 125,
 'crime': 64,
 'adventure crime': 20,
 'drama': 90,
 'thriller': 234,
 'action crime': 4,
 'crime drama': 68,
 'drama thriller': 106,
 'adventure science': 29,
 'animation': 33,
 'family': 109,
 'animation family': 38,
 'fantasy family': 130,
 'action science': 12,
 'adventure action': 17,
 'action thriller': 13,
 'thriller crime': 238,
 'western': 265,
 'adventure western': 32,
 'adventure family': 23,
 'family fantasy': 115,
 'fiction action': 139,
 'action fantasy': 7,
 'comedy': 44,
 'action comedy': 3,
 'comedy science': 59,
 'adventure drama': 22,
 'drama action': 91,
 'romance': 214,
 'drama romance': 104,
 'romance thriller': 228,
 'thriller action': 235,
 'fiction thriller': 150,
 'adventure thriller': 30,
 'fantasy adventure': 126,
 'family adventure': 111,
 'adventure com

In [43]:
print(genre_mat[:3])

  (0, 233)	1
  (0, 135)	1
  (0, 24)	1
  (0, 1)	1
  (0, 138)	1
  (0, 232)	1
  (0, 124)	1
  (0, 16)	1
  (0, 0)	1
  (1, 125)	1
  (1, 24)	1
  (1, 124)	1
  (1, 16)	1
  (1, 0)	1
  (2, 20)	1
  (2, 64)	1
  (2, 1)	1
  (2, 16)	1
  (2, 0)	1


## Cosine Similarity

In [31]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
genre_sim.shape

(4803, 4803)

In [32]:
genre_sim

array([[1.        , 0.59628479, 0.4472136 , ..., 0.        , 0.        ,
        0.        ],
       [0.59628479, 1.        , 0.4       , ..., 0.        , 0.        ,
        0.        ],
       [0.4472136 , 0.4       , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [33]:
#제일 작은 값의 인덱스부터 나열
genre_sim.argsort()

array([[2401, 3037, 3038, ...,  813, 3494,    0],
       [2401, 3067, 3069, ...,  129,    1,  262],
       [2401, 2999, 3000, ..., 1542, 1740,    2],
       ...,
       [   0, 2230, 2229, ..., 1895, 3809, 4800],
       [   0, 3205, 3204, ..., 1596, 1594, 4802],
       [   0, 3141, 3140, ..., 4521, 4710, 4802]], dtype=int64)

In [34]:
genre_sim.argsort()[:,::-1].shape

(4803, 4803)

In [35]:
#-1을 쓰면, reverse로 정렬해서 보여줌
genre_sim.argsort()[:,::-1]

array([[   0, 3494,  813, ..., 3038, 3037, 2401],
       [ 262,    1,  129, ..., 3069, 3067, 2401],
       [   2, 1740, 1542, ..., 3000, 2999, 2401],
       ...,
       [4800, 3809, 1895, ..., 2229, 2230,    0],
       [4802, 1594, 1596, ..., 3204, 3205,    0],
       [4802, 4710, 4521, ..., 3140, 3141,    0]], dtype=int64)

In [36]:
genre_sim_sorted_ind = genre_sim.argsort()[:,::-1]

In [37]:
#0번 레코드의 경우 자기 자신 빼고 3494번, 813번 순으로 유사도가 높다는 뜻
genre_sim_sorted_ind[0]

array([   0, 3494,  813, ..., 3038, 3037, 2401], dtype=int64)

# Recommending movies

In [67]:
def find_sim_movie(df, sorted_ind, title_name, top_n = 10) :
    
    #sorted_ind : 코사인 유사도 index
    #title : 추천 기준이 되는 영화 제목
    # top_n : 추천할 영화 건수
    
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)] #가장 유사한 top_n개
    
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1) #2-D에서 1-D array로
    
    return df.iloc[similar_indexes]

In [66]:
movies_df['title'].head()

0                                      Avatar
1    Pirates of the Caribbean: At World's End
2                                     Spectre
3                       The Dark Knight Rises
4                                 John Carter
Name: title, dtype: object

In [71]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'Avatar', 10)
similar_movies[['title', 'vote_average']]

[[   0 3494  813  870   46   14 1296 1652  419  420]]


,title,vote_average
0,Avatar,7.2
3494,Beastmaster 2: Through the Portal of Time,4.6
813,Superman,6.9
870,Superman II,6.5
46,X-Men: Days of Future Past,7.5
14,Man of Steel,6.5
1296,Superman III,5.3
1652,Dragonball Evolution,2.9
419,Jumper,5.9
420,Hellboy II: The Golden Army,6.5


In [69]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


# improve systems
## votinig수에 따라 가중치 부여.
- using vote_average


$$Weighted Rating  = \frac{v}{v+m} * R + \frac{m}{v+m}*C$$

        - v : 개별 영화에 평점을 투표한 횟수
        - m : 평점을 부여하기 위한 최소 투표 횟수
        - R : 개별 영화에 대한 평균 평점
        - C : 전체 영화에 대한 평균 평점
        
- m값을 높이면 평점 투표 횟수가 많은 영화에 가중치

In [73]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)

print('C : ', round(C,3), 'm : ', round(m,3))

C :  6.092 m :  370.2


In [74]:
percentile = 0.6
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(percentile)

def weighted_vote_average(record) :
    v = record['vote_count']
    R = record['vote_average']
    
    return ((v/(v+m)) * R) + ( (m/(m+v)) * C)

In [76]:
movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis = 1)

In [77]:
movies_df[['title','vote_average','weighted_vote','vote_count']].sort_values('weighted_vote', ascending = False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


## Recommending movies

In [80]:
def find_sim_movie(df, sorted_ind, title_name, top_n = 10) :
    
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    #top_n 의 2배만큼 후보군 추출
    similar_indexes = sorted_ind[title_index, : (top_n * 2)]
    similar_indexes = similar_indexes.reshape(-1)
    
    #기준 영화 인덱스는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending = False)[:top_n]

In [82]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title','vote_average','weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427
